In [13]:
import geopandas as gpd
import pandas as pd
import numpy as np
from pyproj import Proj, CRS, Transformer
from shapely.geometry import Point
from shapely import wkt
import matplotlib.pyplot as plt
import folium
from folium import plugins
import branca
import branca.colormap as cm

plt.rcParams['figure.figsize'] = [15,8]


In [14]:
farm_census =gpd.read_file("../geojson/farm_census/Farm_Census_District_Electoral_Area_2019.geojson")
Parliamentary_constituencies =gpd.read_file("../geojson/boundaries/Parliamentary_Constituencies.geojson")
Council_boundaries =gpd.read_file("../geojson/boundaries/council_boundaries.geojson")
dea_council_excel=pd.ExcelFile("../csv/boundaries/NIMDM17-Tables-DEA2014.xls")
dea_council_csv=pd.read_excel(dea_council_excel, 'Income and Employment')

In [15]:
dea_council_csv["DEA2014"]=dea_council_csv["DEA2014"].apply(str.upper)
dea_council=dea_council_csv.iloc[:, [0,2]]
dea_council.head()



,DEA2014,LGD2014NAME
0,AIRPORT,Antrim and Newtownabbey
1,ANTRIM,Antrim and Newtownabbey
2,BALLYCLARE,Antrim and Newtownabbey
3,DUNSILLY,Antrim and Newtownabbey
4,GLENGORMLEY URBAN,Antrim and Newtownabbey


In [16]:
farm_census_merged= pd.merge(farm_census, dea_council, left_on='DEAName', right_on='DEA2014')
farm_census_merged.drop(["DEAName", "DEACode"], axis=1, inplace=True)
farm_census_merged.insert(1, 'DEA2014', farm_census_merged.pop('DEA2014'))
farm_census_merged.insert(2, 'LGD2014NAME', farm_census_merged.pop('LGD2014NAME'))
farm_census_merged.rename({"AverageSOFarm000":"Average_SOperFarm_Euro000"}, axis =1, inplace=True)
farm_census_merged.head()

,OBJECTID,DEA2014,LGD2014NAME,Year,NumberOfFarms,NumberOflfaFarms,NumberOfNonlfaFarms,TotalAreaFarmedInHa,Average_SOperFarm_Euro000,TotalCrops,...,NumberOfOtherWorkers,TotalNumberOfAgriculturalLabour,NumberOfVSmallFarms,NumberOfSmallFarms,NumberOfMediumFarms,NumberOfLargeFarms,GlobalID,SHAPE_Length,SHAPE_Area,geometry
0,1,AIRPORT,Antrim and Newtownabbey,2019.0,290.0,107.0,183.0,16646.0,98.0,611.0,...,191.0,642.0,203.0,36.0,18.0,33.0,{985B3E14-0DAF-4E44-88D2-170533A4BFCB},1.277837,0.040858,"MULTIPOLYGON (((-6.04202 54.71647, -6.04190 54..."
1,2,ANTRIM,Antrim and Newtownabbey,2019.0,9.0,3.0,6.0,132.0,123.0,4.0,...,3.0,18.0,7.0,3.0,3.0,3.0,{EAC95C9F-A9B8-4F7F-9D7A-8963B521B47C},0.310417,0.003274,"MULTIPOLYGON (((-6.21276 54.74393, -6.21240 54..."
2,3,ARDS PENINSULA,Ards and North Down,2019.0,353.0,6.0,347.0,17059.0,132.0,2720.0,...,310.0,913.0,227.0,43.0,16.0,67.0,{563122F8-44F0-41B9-9EA3-B63A9D376D08},1.386455,0.037509,"MULTIPOLYGON (((-5.56785 54.63039, -5.56782 54..."
3,4,ARMAGH,"Armagh City, Banbridge and Craigavon",2019.0,788.0,402.0,386.0,22431.0,134.0,952.0,...,669.0,1823.0,598.0,99.0,32.0,59.0,{22C22AB0-117C-40EC-8E7C-292D96897C69},1.362793,0.038262,"MULTIPOLYGON (((-6.64085 54.44906, -6.63912 54..."
4,5,BALLYARNETT,Derry City and Strabane,2019.0,17.0,3.0,14.0,1164.0,97.0,385.0,...,18.0,40.0,12.0,3.0,3.0,3.0,{3440FEB8-7491-4E04-8ED3-F13715DECACF},0.344120,0.002721,"MULTIPOLYGON (((-7.25761 55.06650, -7.25757 55..."


In [17]:
farm_census_grouped=farm_census_merged.drop(["OBJECTID", "DEA2014", "GlobalID", "SHAPE_Length", "SHAPE_Area", "geometry"], axis=1).groupby("LGD2014NAME")
farm_census_grouped.first()


,Year,NumberOfFarms,NumberOflfaFarms,NumberOfNonlfaFarms,TotalAreaFarmedInHa,Average_SOperFarm_Euro000,TotalCrops,TotalGrassRGrazingHa,TotalNumberofCattle,TotalNumberofsheep,...,TotalNumberOfPoultry000,NumberOfFarmers,NumberOfSelfEmployed,NumberOfspouses,NumberOfOtherWorkers,TotalNumberOfAgriculturalLabour,NumberOfVSmallFarms,NumberOfSmallFarms,NumberOfMediumFarms,NumberOfLargeFarms
LGD2014NAME,,,,,,,,,,,,,,,,,,,,,
Antrim and Newtownabbey,2019.0,290.0,107.0,183.0,16646.0,98.0,611.0,15847.0,28371.0,26114.0,...,228.0,355.0,222.0,96.0,191.0,642.0,203.0,36.0,18.0,33.0
Ards and North Down,2019.0,353.0,6.0,347.0,17059.0,132.0,2720.0,13944.0,40470.0,18151.0,...,36.0,491.0,296.0,112.0,310.0,913.0,227.0,43.0,16.0,67.0
"Armagh City, Banbridge and Craigavon",2019.0,788.0,402.0,386.0,22431.0,134.0,952.0,21246.0,56304.0,20550.0,...,1145.0,965.0,511.0,189.0,669.0,1823.0,598.0,99.0,32.0,59.0
Belfast,2019.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
Causeway Coast and Glens,2019.0,411.0,209.0,202.0,18547.0,110.0,545.0,17522.0,32009.0,32430.0,...,856.0,534.0,320.0,128.0,196.0,858.0,283.0,53.0,30.0,45.0
Derry City and Strabane,2019.0,17.0,3.0,14.0,1164.0,97.0,385.0,737.0,790.0,1171.0,...,0.0,19.0,13.0,3.0,18.0,40.0,12.0,3.0,3.0,3.0
Fermanagh and Omagh,2019.0,221.0,217.0,3.0,10256.0,60.0,12.0,9799.0,14544.0,11604.0,...,23.0,261.0,140.0,63.0,91.0,415.0,177.0,18.0,12.0,14.0
Lisburn and Castlereagh,2019.0,61.0,5.0,56.0,3978.0,144.0,538.0,3414.0,6991.0,3116.0,...,0.0,93.0,63.0,14.0,83.0,190.0,34.0,8.0,5.0,14.0
Mid Ulster,2019.0,635.0,512.0,123.0,23732.0,48.0,497.0,22979.0,28929.0,72869.0,...,305.0,784.0,372.0,123.0,240.0,1147.0,528.0,60.0,27.0,20.0


In [18]:
farm_census_grouped.sum()

,Year,NumberOfFarms,NumberOflfaFarms,NumberOfNonlfaFarms,TotalAreaFarmedInHa,Average_SOperFarm_Euro000,TotalCrops,TotalGrassRGrazingHa,TotalNumberofCattle,TotalNumberofsheep,...,TotalNumberOfPoultry000,NumberOfFarmers,NumberOfSelfEmployed,NumberOfspouses,NumberOfOtherWorkers,TotalNumberOfAgriculturalLabour,NumberOfVSmallFarms,NumberOfSmallFarms,NumberOfMediumFarms,NumberOfLargeFarms
LGD2014NAME,,,,,,,,,,,,,,,,,,,,,
Antrim and Newtownabbey,14133.0,871.0,463.0,414.0,42358.0,786.0,1517.0,40026.0,77955.0,78599.0,...,1174.0,1100.0,666.0,267.0,506.0,1870.0,622.0,112.0,56.0,94.0
Ards and North Down,14133.0,687.0,72.0,619.0,32320.0,796.0,6384.0,24566.0,64833.0,38908.0,...,158.0,928.0,554.0,221.0,567.0,1714.0,449.0,84.0,44.0,114.0
"Armagh City, Banbridge and Craigavon",14133.0,3245.0,1392.0,1853.0,106854.0,659.0,8994.0,96486.0,242520.0,117824.0,...,3743.0,4064.0,2223.0,882.0,2259.0,7205.0,2418.0,362.0,178.0,287.0
Belfast,18171.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,...,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0
Causeway Coast and Glens,14133.0,2502.0,1550.0,951.0,144007.0,609.0,8231.0,133545.0,184020.0,396514.0,...,2445.0,3258.0,1983.0,751.0,1239.0,5246.0,1750.0,347.0,161.0,247.0
Derry City and Strabane,14133.0,1766.0,1449.0,317.0,97589.0,555.0,4057.0,90601.0,111167.0,283268.0,...,513.0,2211.0,1294.0,472.0,896.0,3578.0,1318.0,225.0,107.0,129.0
Fermanagh and Omagh,14133.0,5094.0,4958.0,134.0,209131.0,414.0,551.0,204514.0,275821.0,279911.0,...,2732.0,6112.0,3327.0,1348.0,1637.0,9098.0,4227.0,488.0,198.0,182.0
Lisburn and Castlereagh,14133.0,841.0,293.0,550.0,35248.0,924.0,4050.0,30725.0,63531.0,38106.0,...,721.0,1072.0,601.0,270.0,529.0,1872.0,609.0,97.0,49.0,92.0
Mid Ulster,14133.0,4144.0,2823.0,1321.0,146431.0,615.0,3453.0,140425.0,262547.0,221704.0,...,9155.0,5103.0,2661.0,950.0,1756.0,7809.0,3235.0,454.0,221.0,234.0
